In [79]:
import pandas as pd
import plotly.express as px

In [107]:
df_trains = pd.read_excel("Inter station database.xls", sheet_name="Runtime", skiprows=1)
df_trains = df_trains.melt(
    id_vars=df_trains.columns[0:5],
    value_vars=df_trains.columns[5:8],
    var_name="Period",
    value_name="Time"
)
df_trains["Speed [m/min]"] = 1000 * df_trains["Distance (Kms)"] / df_trains["Time"]
df_trains["line_id"] = df_trains["Line"].map({
    "Bakerloo ":"bakerloo",
    "Central ":"central",
    "Circle ":"circle",
    "District":"district",
    "East London ":"london-overground",
    "H & C":"hammersmith-city",
    "Jubilee ":"jubilee",
    "Metropolitan":"metropolitan",
    "Northern ":"northern",
    "Piccadilly ":"piccadilly",
    "Victoria":"victoria",
    "Waterloo & City":"waterloo-city"
})
df_trains["mode"] = "tube"
df_trains.loc[df_trains["line_id"]=="london-overground","mode"] = "overground"
for period in df_trains["Period"].unique():
    df_trains = df_trains.append({
        "line_id":"dlr",
        "Speed [m/min]":64*1000/60,
        "mode":"dlr",
        "Period":period
    }, ignore_index=True)
df_trains.head()

,Line,Direction,Station from (A),Station to (B),Distance (Kms),Period,Time,Speed [m/min],line_id,mode
0,Bakerloo,Southbound,HARROW & WEALDSTONE,KENTON,1.74,Un-impeded Running Time (Mins),2.233333,779.104478,bakerloo,tube
1,Bakerloo,Southbound,KENTON,SOUTH KENTON,1.40,Un-impeded Running Time (Mins),1.883333,743.362832,bakerloo,tube
2,Bakerloo,Southbound,SOUTH KENTON,NORTH WEMBLEY,0.90,Un-impeded Running Time (Mins),1.500000,600.000000,bakerloo,tube
3,Bakerloo,Southbound,NORTH WEMBLEY,WEMBLEY CENTRAL,1.27,Un-impeded Running Time (Mins),1.916667,662.608696,bakerloo,tube
4,Bakerloo,Southbound,WEMBLEY CENTRAL,STONEBRIDGE PARK,1.71,Un-impeded Running Time (Mins),2.233333,765.671642,bakerloo,tube


In [108]:
px.box(
    df_trains,
    x="line_id",
    y="Speed [m/min]",
    color="Period"
)

In [109]:
df_trains = df_trains[df_trains["Period"]=="Un-impeded Running Time (Mins)"].groupby(by=["line_id","mode"]) \
    .agg({"Speed [m/min]":"median"}) \
    .reset_index()
display(df_trains)

,line_id,mode,Speed [m/min]
0,bakerloo,tube,526.666667
1,central,tube,767.709438
2,circle,tube,491.883117
3,district,tube,539.318182
4,dlr,dlr,1066.666667
5,hammersmith-city,tube,485.895118
6,jubilee,tube,720.862968
7,london-overground,overground,472.058824
8,metropolitan,tube,664.030189
9,northern,tube,551.363636


In [110]:
df_buses = pd.read_excel(
    "route-all-bus-speeds-to-p02-2020.xlsx",
    sheet_name="2020-21",
    skiprows=25,
    usecols="C,E",
    header=None
)
df_buses.columns = ["line_id","Speed [miles/hr]"]
df_buses["Speed [m/min]"] = df_buses["Speed [miles/hr]"] * 1609.34 / 60
df_buses.drop(inplace=True, columns="Speed [miles/hr]")
df_buses["mode"] = "bus"
df_buses.head()

,line_id,Speed [m/min],mode
0,1,278.392031,bus
1,2,257.608086,bus
2,2U,262.900364,bus
3,3,292.196589,bus
4,4,246.081304,bus


In [111]:
display(df_buses[df_buses["Speed [m/min]"].isna()])
df_buses.dropna(inplace=True)

,line_id,Speed [m/min],mode
447,601,NaN,bus
448,602,NaN,bus
453,625,NaN,bus
457,639,NaN,bus
460,649,NaN,bus
463,669,NaN,bus
464,670,NaN,bus
527,NG2,NaN,bus
558,UL7,NaN,bus


In [112]:
px.histogram(
    df_buses,
    x="Speed [m/min]"
)

In [113]:
df = pd.concat([
    df_trains[["line_id","Speed [m/min]","mode"]],
    df_buses[["line_id","Speed [m/min]","mode"]]
], ignore_index=True)
df.rename(inplace=True, columns={"Speed [m/min]":"speed"})
df.to_csv("../dash_app/common/data_processed/speed_by_line.csv", index=False)